<a href="https://colab.research.google.com/github/nathanzilgo/Data-Science-Machine-Learning/blob/master/LDA_para_tweets_de_parlamentares.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Usar LDA para agrupar tweets de parlamentares em tópicos
> **TODOS**:
* Incrementar as stopwords para os tweets
* Testar valores diferentes no plate notation
* Implementar uma visualização gráfica

In [63]:
import pandas
import numpy as np
import altair
import zipfile 

from sklearn.feature_extraction.text import CountVectorizer
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

import re
from time import time

from google.colab import drive



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [35]:
drive.mount('/content/drive')

zf = zipfile.ZipFile("/content/drive/My Drive/Colab Notebooks/tweets_parlamentares_new.zip", 'r')

raw = pandas.read_csv("./tweets_parlamentares_new.csv", nrows=100000)
zf.close()

tweets = raw

print('Total de tweets: {0}'.format(tweets.shape[0]))
print()
print('Por parlamentar: ')
print()
tweets['nome_eleitoral'].value_counts()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Total de tweets: 100000

Por parlamentar: 



BACELAR                      8478
DANIEL COELHO                6204
BOHN GASS                    5929
ALEXANDRE PADILHA            5410
CORONEL TADEU                4360
                             ... 
AMARO NETO                      9
ALEX SANTANA                    7
ANDRÉ FERREIRA                  6
CHRISTIANE DE SOUZA YARED       4
ÁTILA LIRA                      1
Name: nome_eleitoral, Length: 79, dtype: int64

# Pre-processamento
> Remover algumas stopwords

In [41]:
# Remover stopwords em portugues
stop_words = set(stopwords.words("portuguese"))
stop_words.update(['que', 'até', 'esse', 
                    'essa', 'pro', 'pra',
                    'oi', 'lá', 'blá', 'bb', 
                    'bbm', 'abm', 'cbm', 
                    'dbm', 'dos', 
                    'ltda', 'editora', 'nan', 'https'])

clean_tweets = list()

for w in range(len(tweets.text)):
  tw = str(tweets['text'].iloc[w])

  # remove special characters and digits
  tw  = re.sub("(\\d|\\W)+|\w*\d\w*"," ", tw )
  tw = ' '.join(s for s in tw.split() if (not any(c.isdigit() for c in s)) and len(s) > 2)
  clean_tweets.append(tw)

clean_tweets[20:50]


['nan',
 'https www facebook com AcacioFavacho posts',
 'sessão Congresso Nacional votando vetos presidenciais deliberando junto com parlamentares sobre PLN que autoriza Executivo contratar operação crédito suplementar valor total bilhões',
 'Recebendo Prefeito Clécio Luiz para tratarmos sobre emendas para nossa querida capital Macapá',
 'votação que versa sobre Políticas Sociais Ambientais porém esse novo texto traz inseguranças políticas não consegue assegurar suas implementações por isso PROS vota NÃO pela obstrução dessa matéria pela manutenção texto original Código Ambiental',
 'dia iniciou com muito trabalho Reunião com Presidente Jair Bolsonaro ministro Onyx Lorenzoni bancada PROS',
 'Nossos primeiros dias mandato Obrigado Estado Amapá Foram dias muitas ações Participamos mais frentes parlamentares votações plenário emendas reuniões proposituras projetos lei onde deles propõe isenção IPI',
 'Nossos primeiros dias Obrigado meu Amapá',
 'reunião com bancada PROS qual junto com col

### LDA funciona baseado em frequências de palavras, então usaremos TFs, e não TF-IDFs para os textos dos tweets.

* **max_df**: ignore terms that have a document frequency strictly higher than the given threshold (corpus-specific stop words). If float, the parameter represents a proportion of documents, integer absolute counts.

* **min_df**: ignore terms that have a document frequency strictly lower than the given threshold. This value is also called cut-off in the literature. If float, the parameter represents a proportion of documents, integer absolute counts.

Neste caso, tive resultados interessantes com mindf = 1, max_df = 0.4, max_features= 10000 e ngram_range(1,2)

In [52]:
# Define o "comportamento" do vetorizador
tf_vectorizer = CountVectorizer(
    min_df = 1,
    max_df = 0.4,
    max_features = 10000,
    stop_words = stop_words,
    ngram_range=(1,2)
)

# Aplica o vectorizer para os textos dos tweets
vec_text = tf_vectorizer.fit_transform(clean_tweets)

# Lista de palavras
words = tf_vectorizer.get_feature_names()

# Formato do vetor:
print("Formato do vetor: ", vec_text.shape)
# Quantidade de palavras (features):
print("Quantidade de features: ", len(words))
# visualizar o vetor de features (palavras):
print("Vetor de features: \n", vec_text[0:10])

Formato do vetor:  (100000, 10000)
Quantidade de features:  10000
Vetor de features: 
   (0, 948)	1
  (0, 3427)	1
  (0, 6089)	1
  (0, 621)	1
  (0, 1437)	1
  (0, 9218)	1
  (0, 8377)	1
  (0, 9212)	1
  (0, 736)	1
  (0, 2494)	1
  (0, 3680)	1
  (0, 655)	1
  (0, 237)	1
  (0, 2535)	1
  (0, 2806)	1
  (0, 949)	1
  (0, 9222)	1
  (0, 737)	1
  (0, 241)	1
  (0, 2539)	1
  (1, 948)	1
  (1, 3427)	1
  (1, 6089)	1
  (1, 621)	1
  (1, 1437)	1
  :	:
  (7, 9288)	1
  (7, 8753)	1
  (7, 6566)	1
  (7, 5334)	1
  (7, 7578)	1
  (7, 6183)	1
  (7, 1290)	1
  (7, 8272)	1
  (7, 94)	1
  (7, 1286)	1
  (7, 6582)	1
  (7, 9850)	1
  (7, 3011)	1
  (7, 7546)	1
  (7, 426)	1
  (7, 8667)	1
  (7, 7581)	1
  (8, 3011)	1
  (8, 6282)	1
  (8, 1346)	1
  (8, 467)	1
  (8, 2852)	1
  (8, 7477)	1
  (8, 2854)	1
  (8, 7479)	1


In [53]:
words[0:10]

['__twitter_impression',
 '__twitter_impression true',
 '_rodrigoneves_',
 'abaixo',
 'abaixo assinado',
 'abandona',
 'abandonada',
 'abandonar',
 'abandono',
 'abastecimento']

## Encontrando tópicos

O resultado terá 

*   uma matriz que descreve a relação entre palavras e tópicos
*   uma matriz que descreve a relação entre documentos e tópicos

Existe uma outra implementação de LDA popular em python chamada Gensim.

In [54]:
from sklearn.decomposition import LatentDirichletAllocation

In [67]:
# Utilitários de visualização de dados do Prof Nazareno
def print_top_words(model, feature_names, n_top_words):

    for topic_idx, topic in enumerate(model.components_):

        print("\n--\nTopic #{}: ".format(topic_idx + 1))
        message = ", ".join([feature_names[i]
                              for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)

    print()

def display_topics(W, H, feature_names, documents, no_top_words, no_top_documents):

    for topic_idx, topic in enumerate(H):

        print("\n--\nTopic #{}: ".format(topic_idx + 1))
        print(", ".join([feature_names[i]
                for i in topic.argsort()[:-no_top_words - 1:-1]]).upper())
        
        top_d_idx = np.argsort(W[:,topic_idx])[::-1][0:no_top_documents]
        
        for d in top_d_idx: 
          doc_data = tweets[['nome_eleitoral', 'status_url']].iloc[d]
          print('{} - {} : \t{:.2f}'.format(doc_data[1], doc_data[0], W[d, topic_idx]))

## **Agrupamento (Dúvidas)**

### Atributos do Plate Notation:
* **doc_topic_prior <float, optional (default=None)>**: 
Prior of document topic distribution theta

* **topic_word_prior <float, optional (default=None)>**: Prior of topic word distribution beta


In [57]:
lda = LatentDirichletAllocation(n_components=5, # número de tópicos
                                # doc_topic_prior=5,
                                # topic_word_prior=0.6, # Prior of topic word distribution beta
                                learning_method='online', # 'online' equivale a minibatch no k-means
                                random_state=0) # obter os mesmos resultados aleatórios

t0 = time() 

lda.fit(vec_text) # Learn model from vectorized text

doc_topic_matrix = lda.transform(vec_text) # Matriz de relação docs x topics

print("done in %0.3fs." % (time() - t0))

done in 200.351s.


In [58]:
print('Matriz documento-tópicos:' + str(doc_topic_matrix.shape))
print('Matriz tópicos-termos:' + str(lda.components_.shape))

Matriz documento-tópicos:(100000, 5)
Matriz tópicos-termos:(5, 10000)


In [68]:
# TODO: Tirar dúvida de como os tópicos estão sendo agrupados
display_topics(doc_topic_matrix,
               lda.components_, 
               words,
               tweets,
               15, 
               10)


--
Topic #1: 
SOBRE, CÂMARA, DIA, HOJE, WWW, BOM, DEPUTADOS, COMISSÃO, FEIRA, REUNIÃO, DEBATE, HTTP, DEPUTADO, PROJETO, NESTA
https://twitter.com/AcacioFavacho/status/1096031668400521222 - ACÁCIO FAVACHO : 	0.97
https://twitter.com/ArnaldoJardim/status/1207096766669217792 - ARNALDO JARDIM : 	0.97
https://twitter.com/ArnaldoJardim/status/1190329202891010048 - ARNALDO JARDIM : 	0.97
https://twitter.com/ArnaldoJardim/status/1184454903361687552 - ARNALDO JARDIM : 	0.97
https://twitter.com/ArnaldoJardim/status/1184937238918057999 - ARNALDO JARDIM : 	0.96
https://twitter.com/CarlosZarattini/status/1259817885578539008 - CARLOS ZARATTINI : 	0.96
https://twitter.com/ArnaldoJardim/status/1185626316663611393 - ARNALDO JARDIM : 	0.96
https://twitter.com/christinoaureo/status/1229756343910830087 - CHRISTINO AUREO : 	0.96
https://twitter.com/depaguinaldo11/status/1217539198095437825 - AGUINALDO RIBEIRO : 	0.96
https://twitter.com/depaguinaldo11/status/1166410244504936448 - AGUINALDO RIBEIRO : 	0.96